# LANL initialisation

Skip this notebook - it just collects code shared between notebooks 1+.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import igraph as ig
import re

from __future__ import print_function
from math import log, pow
from collections import Counter

lanl_path = "/Users/etc/data/LANL/"

dnsfile = lanl_path + "dns.txt"
flowfile = lanl_path + "flows.txt"
procfile = lanl_path + "proc.txt"


##############################################################################
# section 2: aligning timestamps

"""
x = proc
y = flow
z = dns

y - x = -20 * 60
z - y =  80 * 60
x - z = -60 * 60 

==> 
"""

flow_offset = -20 * 60
dns_offset = 60 * 60
proc_offset = 0

def flow_parser(x): return  [int(x[0]) - flow_offset, int(x[1])] \
                + [str(x[i]) for i in range(2,6)] \
                + [int(x[i]) for i in range(6,9)]

def dns_parser(x): return [int(x[0]) - dns_offset]\
               + [str(x[i]) for i in range(1,len(x))]

def proc_parser(x):
    n = len(x)
    return  [int(x[0])] + str(x[1]).split('@') + [str(x[i]) for i in range(2,n)]

procs = sc.textFile(procfile)\
    .map(lambda line: proc_parser(line.split(',')))
flows = sc.textFile(flowfile)\
    .map(lambda line: flow_parser(line.split(',')))
dns = sc.textFile(dnsfile)\
    .map(lambda line: dns_parser(line.split(',')))


##############################################################################
# section 4: Process P265

proc_p265 = procs.filter(lambda x: x[4] == 'P265').map(lambda x: x[0:4] + [x[5]])
p265_comps = proc_p265\
            .filter(lambda x: x[4]=='Start')\
            .map(lambda x: (x[3], x[0]))\
            .groupByKey()